## Case study: Scrape website with heavy anti-bot detection

### Introduction


### Target website
[leboncoin.fr](https://www.leboncoin.fr)

### Reference: 
1. How to Use a Proxy with Selenium in Python (2024): https://www.zenrows.com/blog/selenium-proxy

(Remark: this is a very strict website)

### Step 1: Install libraries

In [1]:
%pip install selenium
%pip install selenium-wire
%pip install webdriver-manager
%pip install python-dotenv
%pip install botasaurus
# to inspect what python package imported in this jupyter notebook
%pip install watermark 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from botasaurus_proxy_authentication import add_proxy_options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import urllib.parse
from dotenv import load_dotenv

_ = load_dotenv()

In [3]:
proxy_hostname = os.getenv("PROXY_HOSTNAME")
proxy_username = os.getenv("PROXY_USERNAME")
proxy_password = os.getenv("PROXY_PASSWORD")

In [4]:
class ProxyServer:
    def __init__(self, proxy_username, proxy_password, proxy_hostname):
        self.proxy_username = proxy_username
        self.proxy_password = proxy_password
        self.proxy_hostname = proxy_hostname

    def __call__(self):
        rand_num = random.randint(1, 9999)
        # Reference: https://docs.brightdata.com/api-reference/proxy/rotate_ips
        ## Add -session parameter to this proxy to change IP address every requests is sent
        proxy = f"{self.proxy_username}-session-rand{rand_num}:{self.proxy_password}@{self.proxy_hostname}"
        return f"http://{proxy}"


proxy_servers = ProxyServer(proxy_username, proxy_password, proxy_hostname)
# call proxy_servers() to register a new IP proxy
# >> proxies = proxy_servers() 

In [5]:
base_url = "https://www.leboncoin.fr/_next/data/cgSWtRf7dTSew_ph71GeS/SearchListing/SEO/FiltersListing.json"

for idx in range(1,11):
      params = {"filter1": "u_car_brand--VOLKSWAGEN", 
          "category": "voitures",
          "page": f"p-{idx}"}
      filter_url = base_url + urllib.parse.urlencode(params)
    
      options = Options()
      # add bright data's proxies
      add_proxy_options(options, proxy_servers()) 
      
      # options.add_argument("--headless=new") # uncomment this code if you want to set to headless mode (= no UI)
      driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                              options=options)
      
      driver.get(filter_url)
      time.sleep(5)
      content = driver.page_source
      time.sleep(2)
      content = driver.find_element(by=By.TAG_NAME, value='pre').text
      time.sleep(1)
      parsed_json = json.loads(content)
      print(parsed_json)
      driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"pre"}
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00718C93+51395]
	(No symbol) [0x00685EF1]
	(No symbol) [0x0053E13A]
	(No symbol) [0x005762BB]
	(No symbol) [0x005763EB]
	(No symbol) [0x005AC162]
	(No symbol) [0x00593ED4]
	(No symbol) [0x005AA570]
	(No symbol) [0x00593C26]
	(No symbol) [0x0056C629]
	(No symbol) [0x0056D40D]
	GetHandleVerifier [0x00A96453+3711107]
	GetHandleVerifier [0x00AD583A+3970154]
	GetHandleVerifier [0x00AD0B28+3950424]
	GetHandleVerifier [0x007C9C99+776393]
	(No symbol) [0x006916C4]
	(No symbol) [0x0068C5A8]
	(No symbol) [0x0068C759]
	(No symbol) [0x0067DD80]
	BaseThreadInitThunk [0x75B97BA9+25]
	RtlInitializeExceptionChain [0x770EBDAB+107]
	RtlClearBits [0x770EBD2F+191]


## Computing Environment

In [ ]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z